<a href="https://colab.research.google.com/github/CC-Shivansh-Gupta/logistic-_reg/blob/main/ann_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import copy, math
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing

data = pd.read_csv('updated_air_quality_data.csv')
data.columns = data.columns.str.replace(' ', '_')
data
#print(type(std_data))
#print(std_data.info())

,Unique_ID,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
0,179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
1,179785,Emissions,Density,UHF42,Unknown,Other,2015-01-01,1.2,Good
2,178540,General Pollution,Miles,UHF42,Unknown,Annual Average,2011-12-01,8.6,Good
3,178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
4,823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...,...
18011,816914,General Pollution,Miles,CD,Unknown,Other,2019-01-01,12.9,Moderate
18012,816913,General Pollution,Miles,CD,Unknown,Other,2010-01-01,14.7,Moderate
18013,816872,General Pollution,Miles,UHF42,Unknown,Other,2010-01-01,43.4,Bad
18014,816832,General Pollution,Miles,UHF42,Queens,Other,2010-01-01,65.8,Bad


In [2]:
rows = data.shape[0]
cols = data.shape[1]
print(rows, cols)

18016 9


In [3]:
data.Unique_ID.describe()
print(data.Unique_ID.unique().shape)

(18016,)


In [4]:
if data.Unique_ID.unique().shape[0] == rows:
 data.set_index('Unique_ID', inplace = True)
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
Unique_ID,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
179785,Emissions,Density,UHF42,Unknown,Other,2015-01-01,1.2,Good
178540,General Pollution,Miles,UHF42,Unknown,Annual Average,2011-12-01,8.6,Good
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,Unknown,Other,2019-01-01,12.9,Moderate
816913,General Pollution,Miles,CD,Unknown,Other,2010-01-01,14.7,Moderate
816872,General Pollution,Miles,UHF42,Unknown,Other,2010-01-01,43.4,Bad


In [5]:
rows = data.shape[0]
cols = data.shape[1]
print(rows, cols)

18016 8


In [6]:
#think how to do it on your own
non_numeric_cols = data.select_dtypes(exclude=['number']).columns
cols_arr = np.where(data.columns.isin(non_numeric_cols))[0]
print(cols_arr)

drop_cols = list(data.columns)
drop_cols.pop(cols-1)
print(drop_cols)

'''for i in range(cols):
  data.column_arr[i].describe()'''

[0 1 2 3 4 5 7]
['Name', 'Measure', 'Geo_Type_Name', 'Geo_Place_Name', 'Time_Period', 'Start_Date', 'Data_Value']


'for i in range(cols):\n  data.column_arr[i].describe()'

In [7]:
data.Name.describe()
data.Data_Value.unique()

array([  0.3,   1.2,   8.6, ...,  95.2,  62.3, 123.9])

In [8]:
data = data.replace('Unknown', '0')
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
Unique_ID,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
179785,Emissions,Density,UHF42,0,Other,2015-01-01,1.2,Good
178540,General Pollution,Miles,UHF42,0,Annual Average,2011-12-01,8.6,Good
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,0,Other,2019-01-01,12.9,Moderate
816913,General Pollution,Miles,CD,0,Other,2010-01-01,14.7,Moderate
816872,General Pollution,Miles,UHF42,0,Other,2010-01-01,43.4,Bad


In [9]:
def extraction(column):
  name_arr1 = np.empty(rows, dtype = object)
  for j in range(rows):
    name_arr1[j] = data.iloc[j, column]
  #print(name_arr1)
  unique_names1 = np.unique(name_arr1)
  return name_arr1, unique_names1

In [10]:
def one_hot_label(arr, unique):
  arrofarr_list = []
  for j in range(len(unique)):
    arr_name = np.empty(len(arr), dtype = object)
    for i in range(len(arr)):
      arr_name[i] = arr[i]
    arrofarr_list.append(arr_name)

  arrofarr_tup = tuple(arrofarr_list)
  arrofarr = np.row_stack(arrofarr_tup)
  for j in range(len(unique)):
    for i in range(len(arr)):
      if arrofarr[j][i] == unique[j]:
        arrofarr[j][i] = '1'
      else:
        arrofarr[j][i] = '0'
  #print(arrofarr)
  return arrofarr

In [11]:
arr_arr_list = []
un_arr_list = []

for j in range(len(cols_arr)-1):
  name_arr, unique_names = extraction(cols_arr[j])
  arr_arr_list.append(name_arr)
  un_arr_list.append(unique_names)

'''arr_arr_tup = tuple(arr_arr_list)
arr_arr = np.row_stack(arr_arr_tup)
un_arr_tup = tuple(un_arr_list)
un_arr = np.row_stack(un_arr_tup)'''
#print(arr_arr_list)
#print(un_arr_list)

'arr_arr_tup = tuple(arr_arr_list)\narr_arr = np.row_stack(arr_arr_tup)\nun_arr_tup = tuple(un_arr_list)\nun_arr = np.row_stack(un_arr_tup)'

In [12]:
for j in range(len(cols_arr)-1):
  arrayofarr = one_hot_label(arr_arr_list[j], un_arr_list[j])
  for i in range(len(un_arr_list[j])):
    data.loc[:, un_arr_list[j][i]] = arrayofarr[i]
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category,Asthma-Related,Emissions,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good,0,1,...,0,0,0,0,0,0,0,0,0,0
179785,Emissions,Density,UHF42,0,Other,2015-01-01,1.2,Good,0,1,...,0,0,0,0,0,0,0,0,0,0
178540,General Pollution,Miles,UHF42,0,Annual Average,2011-12-01,8.6,Good,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,0,Other,2019-01-01,12.9,Moderate,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,General Pollution,Miles,CD,0,Other,2010-01-01,14.7,Moderate,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,General Pollution,Miles,UHF42,0,Other,2010-01-01,43.4,Bad,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for j in range(len(drop_cols)):
  data.drop(drop_cols[j], axis = 1, inplace = True)
data

,Air_Quality_Category,Asthma-Related,Emissions,General Pollution,Hospitalizations,Concentration,Density,Miles,Rates,Borough,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,Good,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179785,Good,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178540,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,Moderate,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,Moderate,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,Bad,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data = data.replace('Bad', '0', regex = True)
data = data.replace('Good', '2', regex = True)
data = data.replace('Moderate', '1', regex = True)
data

,Air_Quality_Category,Asthma-Related,Emissions,General Pollution,Hospitalizations,Concentration,Density,Miles,Rates,Borough,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179785,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178540,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
rows = data.shape[0]
cols = data.shape[1]
#print(rows, cols)

In [16]:
train_number = math.floor(data.shape[0]*0.7)+1
print(train_number)
valid_number = math.floor(data.shape[0]*0.15)
print(valid_number)
test_number = math.floor(data.shape[0]*0.15)
print(test_number)
print(train_number + valid_number + test_number)

12612
2702
2702
18016


In [17]:
arr_for_sort = random.sample(range(rows), rows)
print(len(arr_for_sort))
train_arre = []
for j in range(train_number):
  train_arre.append(arr_for_sort[j])
print(len(train_arre))
valid_arre = []
for j in range(train_number, train_number+valid_number):
  valid_arre.append(arr_for_sort[j])
print(len(valid_arre))
test_arre = []
for j in range(train_number+valid_number, rows):
  test_arre.append(arr_for_sort[j])
print(len(test_arre))

18016
12612
2702
2702


In [18]:
def xy_arr(dedobhai, num):
  x_list = []
  for j in range(cols):
    xj = np.zeros(num)
    for i in range(num):
      xj[i] = data.iloc[dedobhai[i], j]
    x_list.append(xj)

  x_tup = tuple(x_list[1:])
  x = np.column_stack(x_tup)
  y = x_list[0]
  return x, y

In [19]:
x_train , y_train = xy_arr(train_arre, train_number)
print(x_train.shape)
print(y_train.shape)
x_valid, y_valid = xy_arr(valid_arre, valid_number)
x_test, y_test = xy_arr(test_arre, test_number)

(12612, 66)
(12612,)


In [20]:
model = Sequential(
    [
        Dense(100, activation = 'sigmoid'),
        Dense(50, activation = 'relu'),
        Dense(25, activation = 'relu'),
        #Dense(30, activation = 'relu'),
        #Dense(15, activation = 'relu'),
        Dense(3, activation = 'linear'),
    ]
)

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x_train, y_train,
    epochs = 25
)

Epoch 1/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.0668
Epoch 2/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9562
Epoch 3/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9246
Epoch 4/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9170
Epoch 5/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9105
Epoch 6/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9180
Epoch 7/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9065
Epoch 8/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9056
Epoch 9/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9003
Epoch 10/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9012
Epoch 11/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8965
Epoch 12/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8928
Epoch 13/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8986
Epoch 14/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8934
Epoch 15/25
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [21]:
model.fit(
    x_valid, y_valid,
    epochs = 25
)

Epoch 1/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9179
Epoch 2/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8995
Epoch 3/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9041
Epoch 4/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8904
Epoch 5/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8893
Epoch 6/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8952
Epoch 7/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8858
Epoch 8/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8878
Epoch 9/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8726
Epoch 10/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8818
Epoch 11/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8850
Epoch 12/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8820
Epoch 13/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8826
Epoch 14/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8860
Epoch 15/25
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8759
Epoch 16/25
85/85 ━

In [22]:
valid_pred = model.predict(x_valid)
#print(valid_prediction)

85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [23]:
def output(predic):
  sm_pred =  tf.nn.softmax(predic).numpy()
  pred = np.argmax(sm_pred, axis = 1)
  return pred

In [24]:
valid_prediction = output(valid_pred)
print(valid_prediction)

[2 0 2 ... 0 2 0]


In [25]:
train_pred = model.predict(x_train)
train_prediction = output(train_pred)
print(train_prediction)

395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[1 2 2 ... 0 2 1]


In [26]:
def corr(arreybhai, y):
  right_wrong = 0
  for j in range(len(arreybhai)):
    if arreybhai[j] == y[j]:
      right_wrong += 1
  print(right_wrong)
  return (right_wrong/len(arreybhai))*100

In [27]:
train_accuracy = corr(train_prediction, y_train)
print(train_accuracy)

6411
50.8325404376784


In [28]:
valid_accuracy = corr(valid_prediction, y_valid)
print(valid_accuracy)

1417
52.442635085122134


In [29]:
test_pred = model.predict(x_test)
test_prediction = output(test_pred)
test_accuracy = corr(test_prediction, y_test)
print(test_accuracy)

85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1369
50.6661732050333


In [30]:
'''arr = np.arange(rows)
np.random.shuffle(arr)
print(arr.shape)
# validation_arr = arr[: 0.7*  len(arr)]'''

'arr = np.arange(rows)\nnp.random.shuffle(arr)\nprint(arr.shape)\n# validation_arr = arr[: 0.7*  len(arr)]'

In [31]:
'''arr_for_sort = []
for j in range(rows):
  arr_for_sort.append(j)
print(arr_for_sort)'''

'arr_for_sort = []\nfor j in range(rows):\n  arr_for_sort.append(j)\nprint(arr_for_sort)'

In [32]:
'''def label(arrey, ooniq):
  for j in range(len(ooniq)):
    for i in range(len(arrey)):
      if arrey[i] == ooniq[j]:
        arrey[i] = j
  return arrey'''

'def label(arrey, ooniq):\n  for j in range(len(ooniq)):\n    for i in range(len(arrey)):\n      if arrey[i] == ooniq[j]:\n        arrey[i] = j\n  return arrey'

In [33]:
'''train_arre = []
temp_num = random.randint(0, rows-1)
k = 0
#for i in range(train_number):
while k < train_number:
  while temp_num in arr_for_sort:
    if temp_num in train_arre:
      temp_num = random.randint(0, rows-1)
    else:
      train_arre.append(temp_num)
      arr_for_sort.remove(temp_num)
      k += 1
      temp_num = random.randint(0, rows-1)
print(len(train_arre))'''

'train_arre = []\ntemp_num = random.randint(0, rows-1)\nk = 0\n#for i in range(train_number):\nwhile k < train_number:\n  while temp_num in arr_for_sort:\n    if temp_num in train_arre:\n      temp_num = random.randint(0, rows-1)\n    else:\n      train_arre.append(temp_num)\n      arr_for_sort.remove(temp_num)\n      k += 1\n      temp_num = random.randint(0, rows-1)\nprint(len(train_arre))'

In [34]:
#data.rename(columns={'Name': 'Name_Emissions'}, inplace = True)
#data['Name_General'] = data.loc[:, 'Name_Emissions']
#data
#data['Name_Emissions'] = data['Name_Emissions'].str.replace('Emissions', '1')
#data
#data.loc[:, 'Name_Hospi'] = name_arrofarr[3]
#data
'''for i in range(len(unique_names)):
  data.loc[:, unique_names[i]] = name_arrofarr[i]
data'''

'for i in range(len(unique_names)):\n  data.loc[:, unique_names[i]] = name_arrofarr[i]\ndata'

In [35]:
'''name_arr = np.empty(rows, dtype = object)
for j in range(rows):
  name_arr[j] = data.loc[j, 'Name']
print(name_arr)
unique_names = np.unique(name_arr)
#unique_names'''

"name_arr = np.empty(rows, dtype = object)\nfor j in range(rows):\n  name_arr[j] = data.loc[j, 'Name']\nprint(name_arr)\nunique_names = np.unique(name_arr)\n#unique_names"